In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
from collections import Counter
from pathlib import Path
from IPython.display import clear_output
import random

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
image_training_path = image_root_path + 'training/'
image_negative_path = image_root_path + 'negative-full'
image_positive_path = image_root_path + 'positive-full'
tables_path = data_path + 'tables/'
image_croped_neg_path = image_training_path + 'negative/' 

df_neg = pd.read_csv(data_path + 'negative_empirical_png.csv')

fix_resolution = [3328,4096]
crop_size = [224, 224]

In [7]:
def get_size_of_image(image_array):
    height = image_array.shape[0]
    width = image_array.shape[1]
    full_resolution = height * width
    diameter = math.sqrt( height ** 2 + width ** 2)
    dark_area = np.sum(image_array == 0)
    return [f'{height} x {width}', [height, width], full_resolution, diameter, dark_area]

def crop_black_part(image_array):
    # Cropping every column that doesn't contain tissue
    not_dark = np.where(image_array != 0, 1, 0)
    tissue_distrib_x = np.sum(not_dark, axis=0) / not_dark.shape[0]
    tissue_distri_y = np.sum(not_dark, axis=1) / not_dark.shape[1]
    is_tissue_x = np.where(tissue_distrib_x > 0.1, 1, 0)
    is_tissue_y = np.where(tissue_distri_y > 0.1, 1, 0)

    first_tissue_x = np.where(is_tissue_x == 1)[0][0]
    last_tissue_x = np.where(is_tissue_x == 1)[-1][-1]
    first_tissue_y = np.where(is_tissue_y == 1)[0][0]
    last_tissue_y = np.where(is_tissue_y == 1)[-1][-1]

    return image_array[first_tissue_y:last_tissue_y,first_tissue_x:last_tissue_x]


def generate_crop_path(index, roi_num):
    return image_croped_pos_path + f"{index}_{roi_num}_cropped"

In [23]:
def crop_negative_array(image_array):
    tissue_array = np.array(crop_black_part(image_array))
    tissue_width = tissue_array.shape[1]
    tissue_height = tissue_array.shape[0]
    
    if crop_size[1] > tissue_width + 1:
        x_offset_1 = 0
        x_offset_2 = 0
    else:
        x_offset_1 = random.randint(0, tissue_width - crop_size[1])
        x_offset_2 = random.randint(0, tissue_width - crop_size[1])
    
    y_half_point = round((tissue_height - crop_size[0]) / 2)
    y_offset_1 = random.randint(0, y_half_point)
    y_offset_2 = random.randint(y_half_point, tissue_height - crop_size[0])
    
    crop_arrays = [tissue_array[y_offset_1:(y_offset_1 + crop_size[0]), x_offset_1:(x_offset_1 + crop_size[1])], tissue_array[y_offset_2:(y_offset_2 + crop_size[0]), x_offset_2:(x_offset_2 + crop_size[1])]]
    
    ret_arrays = []

    for crop_array in crop_arrays:
        tissue_dist = np.sum(np.where((crop_array / 255) > 0.05, 1, 0)) / (crop_size[0] * crop_size[1])
        too_bright = np.sum(np.where((crop_array / 255) > 0.9, 1, 0) / (crop_size[0] * crop_size[1]))
        print(too_bright)
        if too_bright > 0.5:
            print(f"Too much bright area on picture")  
        elif (tissue_dist < 0.5):
            print(f"Too much dark area on picture try to flip the roi horizontally")
        else:
            print(f"Image will be croped brightness {too_bright}")
            ret_arrays.append(crop_array)
        
    return ret_arrays

In [55]:
def crop_negative(idx):
    path = f"{image_croped_neg_path}{idx}"
    image = Image.open(df_neg.loc[idx,'png_path'])
    resize_image_array = np.array(image.resize(fix_resolution))
    tissue_image_array = crop_black_part(resize_image_array)
    crops = crop_negative_array(tissue_image_array)
    for i,crop in enumerate(crops):
        path_crop = f"{path}_{i}.png"
        df_neg_train.loc[len(df_neg_train),:] = df_neg.loc[idx,:]
        df_neg_train.loc[len(df_neg_train) - 1,'negative_index'] = idx
        #df_neg_train.loc[len(df_neg_train)-1,:] = df_neg.loc[idx,:][['empi_anon','acc_anon','side','asses','age_at_study','calc_find','calc_distrib','other_find','num_find','view_pos','eth_desc','study_date_anon','diag_study_date','relative_dcm_path','spot_mag','diag_date_diff','png_path']]
        df_neg_train.loc[len(df_neg_train)-1,'training_path'] = path_crop
        print(f"Saving image {path_crop}")
        im1 = Image.fromarray(crop, mode='L')
        im1.save(path_crop, mode='L')

In [14]:
crop_negative(df_neg.loc[50,:],50)

In [57]:
df_neg_train = pd.DataFrame(columns=df_neg.columns).rename(columns={'Unnamed: 0': 'negative_index'})
for i in range(0,250):
    print(f"Cropping image number {i} / {280}")
    crop_negative(i)

df_neg_train

In [66]:
df_neg_train['label_0'] = 0
df_neg_train = df_neg_train.rename(columns={'training_path':'crop_path'})
df_neg_train['asses'] = 'N'

with open('neg_training.csv','w') as f:
    df_neg_train.to_csv(f)

df_neg_train

In [64]:
df_neg_train